In [10]:
import casadi as ca

# Sample function to get A at a given k (just for illustrative purposes)
def get_A_at_k(k):
    # For demonstration, let's just return a slightly modified A at each k
    return ca.MX(ca.DM([[1.1 + 0.01*k, 0], [0, 0.95]]))

# Define the initial system matrices (you can modify this as needed)
A = ca.MX(ca.DM([[1.1, 0], [0, 0.95]]))
B = ca.MX(ca.DM([[0], [1]]))

# Prediction horizon
N = 10

# Create the list of A matrices for each sample up to the horizon
As_list = [get_A_at_k(k) for k in range(N)]

# Initialize A_bar and B_bar
num_states = A.size1() 
num_controls = B.size2()
A_bar = ca.MX.zeros(num_states * N, num_states)
B_bar = ca.MX.zeros(num_states * N, num_controls * N)

# function to help create B_bar. It may be too memory inefficient later
import casadi as ca

def multiply_list_in_reverse(matrices):
    """
    Multiplies matrices in the provided list in reverse order.
    
    Args:
    - matrices (list): List of matrices (either CasADi or NumPy).

    Returns:
    - Matrix product in reverse order of the provided list.
    """
    
    if not matrices:  # Check if the list is empty
        return None

    result = matrices[-1]  # Starting with the last matrix in the list
    for i in range(len(matrices) - 2, -1, -1):  # Moving in reverse order
        result = ca.mtimes(matrices[i], result)

    return result


# Construct A_bar and B_bar
for i in range(N): # i = take 2 for example 
    Ak = ca.MX.eye(num_states)  # Start with identity matrix
    for k in range(i+1):  # Multiply by A's from the list to simulate the effect
        Ak = ca.mtimes(As_list[i-k], Ak)

    A_bar[num_states*i:num_states*(i+1), :] = Ak

    # Construct B_bar using the multiply_matrices_in_reverse function 
    # j = 2,1
    for j in range(i,0,-1): 
        # for j = 0, calling As_list[1,2] = A1,A2 
        matrices_to_multiply = As_list[i-j+1:i]
        # multiply_matrices_in_reverse(matrices_to_multiply, 0, 2) = A1
        prod = multiply_list_in_reverse(matrices_to_multiply)
        # Since we didn't append B in the list, we multiply it separately here.
        prod = ca.mtimes(prod, B)
        B_bar[num_states*i:num_states*(i+1), num_controls*j:num_controls*(j+1)] = prod

# Now A_bar and B_bar are constructed and ready for use in MPC formulations.


IndexError: list index out of range